In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam

import seaborn as sns

2024-02-26 20:45:47.698887: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 20:45:48.117174: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 20:45:48.117271: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 20:45:48.180681: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-26 20:45:48.340044: I tensorflow/core/platform/cpu_feature_guar

In [2]:
data_folder = "sequence"
real_folder = os.path.join(data_folder, "REAL")
fake_folder = os.path.join(data_folder, "FAKE")


In [3]:
real_data = [np.load(os.path.join(real_folder, file)) for file in os.listdir(real_folder)]
fake_data = [np.load(os.path.join(fake_folder, file)) for file in os.listdir(fake_folder)]

# Create labels
real_labels = np.zeros(len(real_data))
fake_labels = np.ones(len(fake_data))

# Combine real and fake data
all_data = np.concatenate((real_data, fake_data), axis=0)
all_labels = np.concatenate((real_labels, fake_labels), axis=0)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(all_data, all_labels, test_size=0.2, random_state=42)
X_train = X_train.reshape(-1, 5, 224, 224, 3)  # Assuming 5 timesteps per video
X_test = X_test.reshape(-1, 5, 224, 224, 3)

In [9]:
X_test.shape, y_test.shape

((1, 5, 224, 224, 3), (1,))

In [7]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add 3D convolutional layers for temporal processing
x = base_model.output
x = layers.TimeDistributed(base_model)(x)  # Apply base model to each frame

# Add custom layers for classification
x = layers.GlobalAveragePooling3D()(x)
x = layers.Dense(1, activation='sigmoid')(x)  # Output layer for binary classification

# Compile the model
model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


# Plot training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

# Evaluate model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5, 5))
sns.heatmap(cm, annot=True, fmt="d")
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Plot loss history
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

#test accuracy
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
print(test_acc)

ValueError: Exception encountered when calling layer "time_distributed_1" (type TimeDistributed).

Input 0 of layer "efficientnetb0" is incompatible with the layer: expected shape=(None, 224, 224, 3), found shape=(None, 7, 1280)

Call arguments received by layer "time_distributed_1" (type TimeDistributed):
  • inputs=tf.Tensor(shape=(None, 7, 7, 1280), dtype=float32)
  • training=None
  • mask=None